# ResNet Detector Developement 1

WNixalo - 1:4/5/2018

---

The purpose of this notebook is to experiment with small datasets (CPU dev) and finetune the general-purpose base object detector on the GLoC dataset.

## Imports

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [9]:
from fastai.conv_learner import *

sys.path.insert(1, os.path.join('../'))
from utils import common
from utils import temp_utils
from utils.subfolder_val_idxs import set_val_idxs

In [10]:
df.sample()

NameError: name 'df' is not defined

In [23]:
df.describe()

,gloc
count,7637.000000
mean,0.299201
std,0.457938
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [16]:
cpu_df = temp_utils.get_random_sample(df, p=1000)

In [26]:
cpu_df.describe()

,gloc
count,1000.000000
mean,0.304000
std,0.460213
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [25]:
cpu_df['id']

Index(['id', 'gloc'], dtype='object')

In [40]:
int_df = pd.read_csv(PATH_CSV_BBOX, header=None)

In [41]:
int_df.head()

,0,1,2,3,4,5
0,interstage_train/000000-000412/000000.jpg,83,72,191,380,pilot
1,interstage_train/000000-000412/000001.jpg,52,89,204,381,pilot
2,interstage_train/000000-000412/000002.jpg,58,89,208,390,pilot
3,interstage_train/000000-000412/000003.jpg,66,98,214,388,pilot
4,interstage_train/000000-000412/000004.jpg,65,90,209,389,pilot


In [42]:
int_df.columns = ['id','x1','y1','x2','y2','class']

In [43]:
int_df.head()

,id,x1,y1,x2,y2,class
0,interstage_train/000000-000412/000000.jpg,83,72,191,380,pilot
1,interstage_train/000000-000412/000001.jpg,52,89,204,381,pilot
2,interstage_train/000000-000412/000002.jpg,58,89,208,390,pilot
3,interstage_train/000000-000412/000003.jpg,66,98,214,388,pilot
4,interstage_train/000000-000412/000004.jpg,65,90,209,389,pilot


In [45]:
int_df.to_csv(PATH_CSV_BBOX, index=False)

In [46]:
int_df = pd.read_csv(PATH_CSV_BBOX)

In [47]:
int_df.head()

,id,x1,y1,x2,y2,class
0,interstage_train/000000-000412/000000.jpg,83,72,191,380,pilot
1,interstage_train/000000-000412/000001.jpg,52,89,204,381,pilot
2,interstage_train/000000-000412/000002.jpg,58,89,208,390,pilot
3,interstage_train/000000-000412/000003.jpg,66,98,214,388,pilot
4,interstage_train/000000-000412/000004.jpg,65,90,209,389,pilot


## Data

In [11]:
PATH = Path('../data')
PATH_CSV = PATH/'labels.csv'
PATH_CSV_BBOX = PATH/'interstage_labels.csv'
PATH_DATA = PATH/'train'
PATH_DATA_BBOX = PATH/'interstage_train'

CPU_PATH = PATH/'cpu'
CPU_PATH_CSV = CPU_PATH/'cpu_labels.csv'
CPU_PATH_CSV_BBOX = CPU_PATH/'cpu_interstage_labels.csv'
CPU_PATH_DATA= CPU_PATH/'train'
CPU_PATH_BBOX= CPU_PATH/'intersetage_train'

In [12]:
list(PATH.iterdir())

[PosixPath('../data/labels.csv'),
 PosixPath('../data/.DS_Store'),
 PosixPath('../data/interstage_labels.csv'),
 PosixPath('../data/labels'),
 PosixPath('../data/test_z'),
 PosixPath('../data/interstage_train'),
 PosixPath('../data/train')]

## CPU Sample

In [13]:
temp_utils.create_cpu_dataset(PATH, p=1000)

In [20]:
df = pd.read_csv(PATH_CSV)
df_cpu = pd.DataFrame(columns=['id','gloc'])

In [159]:
path = Path('data/train')
path /= 'test'

In [200]:
def create_cpu_labels(cpu_path, cpu_label_path='cpu_labels', subdir='train'):
    """
        Creates labels from cpu data subset by looking at its `subdir` directory.
            (default: 'train')
    """
##### ensure cpu label filepath correct
    if '.csv' not in cpu_label_path: cpu_label_path += '.csv'
    assert cpu_label_path[-4:] == '.csv'
##### get idxs of cpu dataset files (int(id) = idx in GLoC dataset)
    keep_ids = []
    cpu_data_path = cpu_path /  subdir
    subfolders = os.listdir(cpu_data_path)
    for undes in ['.DS_Store', cpu_label_path.split('/')[-1]]: # remove old label file & .DS_Store
        if undes in subfolders: subfolders.remove(undes)
    subfolders.sort()
    for subfolder in subfolders:
        fnames = os.listdir(cpu_data_path/subfolder)
        fnames.sort()
        for fname in fnames:
            keep_ids.append(subfolder + '/' + fname.split('.')[0])
##### get ids of cpu dataset files
    keep_idxs = [int(i.split('/')[-1]) for i in keep_ids]
    df_cpu = df.loc[keep_idxs]
##### write csv & return df
    df_cpu.to_csv(cpu_path / cpu_label_path, index=False)
    return df_cpu

In [201]:
CPU_PATH_DATA

PosixPath('../data/cpu/train')

In [202]:
df_cpu = create_cpu_labels(CPU_PATH)

In [203]:
df_cpu.head()

,id,gloc
18,000000-000412/000018,0
20,000000-000412/000020,0
21,000000-000412/000021,0
22,000000-000412/000022,0
31,000000-000412/000031,0


In [204]:
temp_utils.count_files(CPU_PATH_DATA)

1000

#### Checking my indices line up w/ dataframe rows

In [90]:
keep_idxs[:10]

[18, 20, 21, 22, 31, 36, 40, 45, 61, 64]

In [68]:
# i = 0
# df.loc[i]
# df.loc[i]['id']
# int(df.loc[i]['id'].split('/')[-1])

id      000000-000412/000000
gloc                       0
Name: 0, dtype: object

In [84]:
for i in range(len(df.index)):
    if i != int(df.loc[i]['id'].split('/')[-1]):
        print(i)
        break

### Validation Set (CPU)

Val Set will be contents of random subfolders.

In [205]:
CPU_PATH_CSV

PosixPath('../data/cpu/cpu_labels.csv')

In [207]:
val_idxs = set_val_idxs(CPU_PATH_CSV)
len(val_idxs)

188

In [208]:
CPU_PATH_CSV

PosixPath('../data/cpu/cpu_labels.csv')

In [209]:
df_cpu.index

Int64Index([  18,   20,   21,   22,   31,   36,   40,   45,   61,   64,
            ...
            7569, 7570, 7575, 7576, 7580, 7589, 7624, 7625, 7626, 7634],
           dtype='int64', length=1000)

As they should be, the CPU dataset contains a random assortment of 1,000 images uniformly sampled from the original dataset. The validation set contains the ids of ~20% of the training set, segregated by subfolder to prevent 'data leakage'.

In [ ]:
sz = 224
bs = 32
f_model = resnet34

tfms = tfms_from_model(f_model, sz=sz, bs=bs, aug_tfms=transforms_side_on, max_zoom=1.1, pad=sz//8, tfm_y=)
model_data = ImageClassifierData.from_csv(CPU_PATH_CSV, CPU_PATH, csv_fname='cpu_labels.csv',
                                          bs=32, tfms=tfms, val_idxs=val_idxs)